# Check Environment and Run Training

In [1]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO
from customenv import PharmacophoreEnv
from wandb.integration.sb3 import WandbCallback
import wandb
import time
from stable_baselines3.common.monitor import Monitor
import gymnasium as gym
from gymnasium.envs.registration import register
from gymnasium.wrappers import FlattenObservation
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
import subprocess
from datetime import datetime

In [16]:
import numpy as np
np.random.uniform(1,5,2)

array([1.09219097, 3.13685559])

In [2]:
register(
    # unique identifier for the env `name-version`
    id="PharmacophoreEnv-v0",
    # path to the class for creating the env
    entry_point="customenv:PharmacophoreEnv",
    max_episode_steps=200,
    # Max number of steps per episode, using a `TimeLimitWrapper`
    kwargs={"output": r'C:\\Users\\kilia\\MASTER\\rlpharm\\data\\hitlists\\hitlist', 
            "querys": r'C:\\Users\\kilia\\MASTER\\rlpharm\\data\\querys\\sEH-1ZD5_mod5_LS_3.02.pml', 
            "actives_db": r'C:\\Users\\kilia\\MASTER\\rlpharm\\data\\ldb2s\\actives.ldb2',
            "inactives_db": r"C:\\Users\\kilia\\MASTER\\rlpharm\\data\\ldb2s\\inactives.ldb2",
            "approximator": r"C:\Users\kilia\MASTER\rlpharm\data\models\approximator\best.pt",
            "ldba": 58,
            "ldbi": 177,
            "features": "H,HBA,HBD",
            "enable_approximator": False},
)

env = gym.make("PharmacophoreEnv-v0")
env.generate_examples(n=10000, csv_file=r"C:\Users\kilia\MASTER\rlpharm\data\approx.csv")



KeyboardInterrupt: 

In [ ]:
import numpy as np
b = np.array([1, 5], dtype=np.float32)
x = np.array([1,1,1,1], dtype=np.float32)
y = np.array([1,0.9,1,1], dtype=np.float32)
q = {'a':x, 'b':y}
z = not np.logical_and(np.all(q['a'] <= b[1]), np.all(q['a'] >= b[0]))
m = not np.logical_and(np.all(q['b'] <= b[1]), np.all(q['b'] >= b[0]))

print(z)
print(m)

if z or m:
    print("y is true")



In [2]:
output= r'C:\\Users\\kilia\\MASTER\\rlpharm\\data\\hitlists\\hitlist'
querys= r'C:\\Users\\kilia\\MASTER\\rlpharm\\data\\querys\\sEH-1ZD5_mod5_LS_3.02.pml'
actives_db= r'C:\\Users\\kilia\\MASTER\\rlpharm\\data\\ldb2s\\actives_mini.ldb2:active'
inactives_db= r"C:\\Users\\kilia\\MASTER\\rlpharm\\data\\ldb2s\\inactives_mini.ldb2:inactive"
approximator= r"C:\Users\kilia\MASTER\rlpharm\data\models\approximator\best.pt"

In [8]:
def exec_vhts(output_file, querys, actives_db, inactives_db):
    """
    Execute VHTS with the given querys
    :param querys: path to query file
    :param actives_db: path to actives database
    :param inactives_db: path to inactives database
    :return: count of positive and negative hits as separate values
    """
    datestring = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%f")
    output_file = output_file + '-{}.sdf'.format(datestring)  
    arguments = ["--query", querys, "--database", f"{actives_db}, {inactives_db}", "--output", output_file]
    command = r'C:/Users/kilia/MASTER/rlpharm/LS5/iscreen.sh' + " " + " ".join(arguments)
    print(command)
    #subprocess.run(["bash", r'C:\Users\kilia\MASTER\rlpharm\LS5\iscreen.sh'] + arguments, shell=True)
    subprocess.run(['bash', command], shell=True)
    poshits = 0
    neghits = 0
    with open(file= output_file.replace("\\\\", "\\") , mode='r') as f:
        for line in f:
            if line.startswith('active'):
                poshits += 1
            if line.startswith('decoy'):
                neghits += 1
    print("Hits(pos: " + str(poshits) + ", neg: " + str(neghits) + ")")
    return poshits, neghits

In [9]:
pos, neg = exec_vhts(output, querys, actives_db, inactives_db)

C:/Users/kilia/MASTER/rlpharm/LS5/iscreen.sh --query C:\\Users\\kilia\\MASTER\\rlpharm\\data\\querys\\sEH-1ZD5_mod5_LS_3.02.pml --database C:\\Users\\kilia\\MASTER\\rlpharm\\data\\ldb2s\\actives_mini.ldb2:active, C:\\Users\\kilia\\MASTER\\rlpharm\\data\\ldb2s\\inactives_mini.ldb2:inactive --output C:\\Users\\kilia\\MASTER\\rlpharm\\data\\hitlists\\hitlist-2023_05_14-03_53_33_741759.sdf


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\kilia\\MASTER\\rlpharm\\data\\hitlists\\hitlist-2023_05_14-03_53_33_741759.sdf'